In [1]:
# Websites:
# 1. https://www.rent.com/california/los-angeles-apartments/axis-4-100064358
# Scrape data of each available unit.

In [2]:
# Key Data Features:
# - Address and unit number
# - Sq ft
# - Type (studio, home, apartment)
# - Beds
# - Baths
# - Date available
# - Asking rent

In [3]:
! pip install fuzzywuzzy

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
! pip install beautifulsoup4

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
! pip install selenium

You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [39]:
from fuzzywuzzy import fuzz
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re
from random import randint
pd.set_option('display.max_rows', 500)
import numpy as np


In [7]:
# Install homebrew on mac
# Follow Instructions from https://medium.com/dropout-analytics/selenium-and-geckodriver-on-mac-b411dbfe61bc
# Install geckodriver via homebrew
# Copy Paste "/usr/local/bin/geckodriver" file path. This should remain the same, but in case it does not you 
# you will need to replace every executable_path variable

In [8]:
#confirm geckodriver is in list below
! ls

LinkedIn Crawler_3262020.ipynb        rent_com_get_apartment_details.ipynb
geckodriver.log                       rent_com_get_apt_links_for_city.ipynb
los_angeles_apartments_details.csv    webdriver
los_angeles_apartments_urls.csv


In [23]:
apartment_df = pd.read_csv('los_angeles_apartments_urls.csv')
apartment_df.head()

,Unnamed: 0,0
0,0,https://www.rent.com//california/burbank-apart...
1,0,https://www.rent.com//california/los-angeles-a...
2,0,https://www.rent.com//california/hollywood-apa...
3,0,https://www.rent.com//california/glendale-apar...
4,0,https://www.rent.com//california/burbank-apart...


In [24]:
apartment_info_already_scraped = pd.read_csv('los_angeles_apartments_details.csv')
apartment_info_already_scraped = apartment_info_already_scraped['url_link'].unique()
links_not_yet_scraped = (set(apartment_df['0'].tolist()) - set(apartment_info_already_scraped))
all_links = apartment_df['0'].tolist()
links_not_yet_scraped = list(links_not_yet_scraped)

In [25]:
len(links_not_yet_scraped)

0

In [18]:
len(apartment_df)

4676

In [19]:
list_of_apartment_urls = links_not_yet_scraped

In [20]:
for page_url in list_of_apartment_urls:
    
    start_time = time.time()

    time.sleep(2)

    options = webdriver.FirefoxOptions()
    options.add_argument("start-maximized")
    options.add_argument('disable-infobars')

    page_url = page_url

    browser = webdriver.Firefox(options=options, 
                               executable_path='/usr/local/bin/geckodriver')

    browser.get(page_url)
    result=BeautifulSoup(browser.page_source)

    description = result.findAll('td', {'data-tid': 'pdpfloorplan-displayText'})
    price = result.findAll('td', {'data-tid': 'pdpfloorplan-price'})
    bedbaths = result.findAll('td', {'data-tid': 'pdpfloorplan-bedbaths'})
    sqft = result.findAll('td', {'data-tid': 'pdpfloorplan-sqft'})
    available = result.findAll('td', {'data-tid': 'pdpfloorplans-unavailable'})

    browser.quit()

    description_text = []
    for text_ in description:
        description_text.append(text_.text)

    price_text = []
    for text_ in price:
        price_text.append(text_.text)

    bedbaths_text = []
    for text_ in bedbaths:
        bedbaths_text.append(text_.text)

    sqft_text = []
    for text_ in sqft:
        sqft_text.append(text_.text)

    available_text = []
    for text_ in available:
        available_text.append(text_.text)

    details = pd.DataFrame([description_text,price_text,bedbaths_text,sqft_text,available_text]).T
    details.columns = ['description', 'price', 'bedbaths','sqft','available']
    
    if len(details) == 0:
        details = pd.DataFrame([['','','','','']])
        details.columns = ['description', 'price', 'bedbaths','sqft','available']
    else:
        details = details

    try:
        details['bedrooms_extract'] = details['bedbaths'].str.extract(r'((Studio)|((\d+) Bed))', expand=False)[0]
    except:
        details['bedrooms_extract'] = "na"
        
    try:
        details['bathrooms_extract'] = details['bedbaths'].str.extract(r'(((\d+) Bath)|((\d+) Baths))', expand=False)[0]
    except:
        details['bathrooms_extract'] = "na"
    
    try:
        details['price_extract'] = details['price'].str.extract(r'([0-9]+(,[0-9]+)+)', expand=False)[0]
    except:
        details['price_extract'] = "na"
        
    try:
        details['sqft_extract'] = details['sqft'].str.extract(r'([0-9]+)', expand=False)
    except:
        details['sqft_extract'] = "na"

    try:
        address = result.findAll('div', {'data-tid': 'pdpKeyInfo_citystatezip'})[0]
        address = address.text
    except:
        address = "na"
    
    
    try: 
        neighborhood = result.findAll('div', {'data-tid': 'pdpKeyInfo_neighborhood'})[0]
        neighborhood = neighborhood.text
    except:
        neighborhood = "na"

    details['address'] = address
    details['neighborhood'] = neighborhood
    details['url_link'] = page_url
    details['scrape_date'] = pd.to_datetime('today')
    
    city = 'los-angeles-apartments'.replace('-','_')

    if all_links.index(page_url) == 0:
        details.to_csv('%s_details.csv' % city)
    else: 
        with open('%s_details.csv' % city, 'a') as f:
            details.to_csv(f, header=False)



/Users/bryceeb/anaconda/envs/python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/bryceeb/anaconda/envs/python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [ ]:
# data.reset_index(inplace=True)
# data_dict = data.to_dict("records")
# company.insert_one({"index":"Sensex","data":data_dict})